# BERTopic Tutorial
Grootendorst, M. (2022). BERTopic: Neural topic modeling with a class-based TF-IDF procedure. arXiv preprint arXiv:2203.05794.

https://arxiv.org/pdf/2203.05794.pdf


# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

# Data
For this example, we use a large Kaggle dataset with 20 topics for news headlines from Huffington Post.

In [2]:
import pandas as pd
url = 'https://github.com/herbertfreeze/COMM557/raw/master/tutorial_5_advanced_topic_modeling/news_headline_sample.csv'
df  = pd.read_csv(url)
df = df[~df.headline.isna()]
df = df.sample(frac = 0.1)
docs = df.headline.to_list()

In [3]:
df.category.value_counts()

POLITICS          3549
WELLNESS          1755
ENTERTAINMENT     1725
STYLE & BEAUTY     990
TRAVEL             972
PARENTING          911
FOOD & DRINK       654
HEALTHY LIVING     628
QUEER VOICES       626
BUSINESS           613
COMEDY             563
SPORTS             503
BLACK VOICES       471
THE WORLDPOST      405
HOME & LIVING      403
WEDDINGS           385
CRIME              373
WOMEN              368
PARENTS            367
IMPACT             348
DIVORCE            342
WORLD NEWS         300
MEDIA              291
RELIGION           281
GREEN              274
WORLDPOST          274
WEIRD NEWS         268
STYLE              234
SCIENCE            216
TECH               205
TASTE              202
MONEY              176
U.S. NEWS          156
ARTS               154
ENVIRONMENT        146
ARTS & CULTURE     145
GOOD NEWS          135
FIFTY              134
COLLEGE            114
LATINO VOICES      104
CULTURE & ARTS      96
EDUCATION           96
Name: category, dtype: int64

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [4]:
%%time
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/655 [00:00<?, ?it/s]

2023-10-19 20:18:57,380 - BERTopic - Transformed documents to Embeddings
2023-10-19 20:19:39,163 - BERTopic - Reduced dimensionality
2023-10-19 20:21:17,133 - BERTopic - Clustered reduced embeddings


CPU times: user 2min 57s, sys: 4.13 s, total: 3min 1s
Wall time: 3min 20s


## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [5]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9793,-1_the_in_to_and,"[the, in, to, and, photos, of, for, is, trump,...",[9 Things To Make The Most Out Of Your High Sc...
1,0,404,0_gay_transgender_lgbt_queer,"[gay, transgender, lgbt, queer, lgbtq, marriag...",[The Gay Culture Every Queer Should Know About...
2,1,355,1_recipes_recipe_cake_cuff,"[recipes, recipe, cake, cuff, chocolate, cooki...",[Sylvia Plath's Surprisingly Delicious Tomato ...
3,2,301,2_gun_shooting_police_officer,"[gun, shooting, police, officer, shot, cop, te...",[Support For Gun Control Spikes After Orlando ...
4,3,189,3_wedding_weddings_brides_bride,"[wedding, weddings, brides, bride, ring, guest...","[How to Have the Wedding of Your Dreams, 'Four..."


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [6]:
topic_model.get_topic(0)  # Select the most frequent topic

[('gay', 0.053378950103415125),
 ('transgender', 0.024764217280954712),
 ('lgbt', 0.021530296443038832),
 ('queer', 0.019283143767127857),
 ('lgbtq', 0.017771022028902905),
 ('marriage', 0.01764536828609453),
 ('trans', 0.015717707928410308),
 ('equality', 0.015687139102606763),
 ('samesex', 0.015322554243545389),
 ('rights', 0.013514464377325225)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [7]:
topic_model.topics_[:10]

[-1, -1, 152, 261, 45, 2, -1, 93, -1, 27]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [8]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [9]:
topic_model.visualize_distribution(probs[200], min_probability=0.005)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [10]:
topic_model.visualize_barchart(top_n_topics=5)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [11]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [12]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('gay', 0.03695647280194992),
 ('transgender', 0.015445514300470618),
 ('lgbt', 0.013286423454642293),
 ('queer', 0.011916327585476451),
 ('marriage', 0.011905022677331116),
 ('lgbtq', 0.010852763936069114),
 ('trans', 0.009863241672451254),
 ('equality', 0.009638842433267899),
 ('gay marriage', 0.009553659173234042),
 ('samesex', 0.009156857439972634)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [13]:
topic_model.reduce_topics(docs, nr_topics=60)

2023-10-19 20:21:28,492 - BERTopic - Reduced number of topics from 313 to 60


In [14]:
# Access the newly updated topics with:
print(topic_model.topics_)

[-1, -1, 49, 3, 1, 8, -1, 1, -1, 20, 8, 1, -1, -1, 1, 25, 0, -1, 4, -1, 0, 41, 18, 1, 7, -1, 4, 5, -1, -1, 7, 15, -1, -1, -1, 23, -1, -1, 2, 5, -1, 5, 0, -1, 11, 28, 3, 26, 17, -1, -1, 0, -1, -1, 21, 0, -1, 4, 28, 5, -1, -1, -1, 13, 11, 34, -1, 5, 17, 17, 19, -1, 3, -1, 18, 18, 4, 30, -1, -1, 10, 13, -1, 9, -1, -1, -1, 4, -1, -1, 6, 21, 8, 5, 13, -1, 6, -1, 11, 14, -1, -1, 4, 6, -1, -1, 7, 1, -1, -1, 23, -1, 0, 0, 0, -1, 5, 17, 30, 4, -1, 5, 6, -1, 8, 30, 33, 26, 19, 43, 9, 2, -1, -1, 24, -1, -1, 0, -1, 9, 29, -1, -1, 1, -1, -1, -1, 3, 4, -1, -1, 45, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 9, -1, -1, 2, -1, 21, -1, 16, -1, 2, -1, 8, -1, 2, -1, 16, 3, -1, 7, 2, -1, 13, -1, 12, -1, -1, 19, 37, 1, -1, -1, -1, -1, 1, 0, -1, -1, -1, -1, 46, 5, -1, 2, -1, -1, -1, 14, -1, -1, -1, -1, 47, -1, 5, 28, 0, 4, 7, 1, 5, -1, -1, 2, -1, 45, 2, -1, -1, -1, 49, -1, 5, 15, -1, 25, -1, -1, -1, 35, -1, 1, -1, 4, -1, 12, -1, -1, -1, 3, -1, -1, 5, -1, -1, -1, -1, -1, 23, -1, 0, -1, 1, 7, -1, 21, -1, 4

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [15]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[43, 11, 8, 2, 13]

In [16]:
topic_model.get_topic(71)

False

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [17]:
# Save model
topic_model.save("my_model")

In [18]:
# Load model
my_model = BERTopic.load("my_model")

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
